In [1]:
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file("../../msds434-whoop-app-44384939c1f4.json")

project_id = 'msds434-whoop-app'
client = bigquery.Client(credentials=credentials, project=project_id)

In [6]:
# daily data
query_job = client.query(
    """ 
    SELECT

r.cycle_id
, r.sleep_id
, date(r.created_at) recovery_date
, extract(week from r.created_at at TIME ZONE 'America/Chicago') as week_of_year
, extract(dayofweek from r.created_at at TIME ZONE 'America/Chicago') as day_of_week
, r.score_state
, r.score_recovery_score as recovery_score
, r.score_resting_heart_rate as resting_heart_rate
, r.score_hrv_rmssd_milli as hrv_milli
, r.score_spo2_percentage as spo2_perc
, r.score_skin_temp_celsius as skin_temp_celsius
, s.score_state
, s.start as sleep_start
, s.end as sleep_end
, case when extract(hour from s.start AT TIME ZONE 'America/Chicago') < 12
 then extract(hour from s.start AT TIME ZONE 'America/Chicago') + 24
 else extract(hour from s.start AT TIME ZONE 'America/Chicago' ) end +
 extract(minute from s.start AT TIME ZONE 'America/Chicago')/60.0 as sleep_start_time
, s.score_stage_summary_total_in_bed_time_milli as in_bed_time
, s.score_stage_summary_total_no_data_time_milli as no_data_time
, s.score_stage_summary_total_light_sleep_time_milli as light_sleep_time
, s.score_stage_summary_total_slow_wave_sleep_time_milli as slow_wave_sleep_time
, s.score_stage_summary_total_rem_sleep_time_milli as rem_sleep_time
, s.score_stage_summary_sleep_cycle_count as sleep_cycle_count
, s.score_stage_summary_disturbance_count as disturbance_count
, s.score_sleep_needed_baseline_milli as sleep_need
, s.score_sleep_needed_need_from_sleep_debt_milli as sleep_need_from_debt
, s.score_sleep_needed_need_from_recent_strain_milli as sleep_need_from_strain
, s.score_respiratory_rate as respiratory_rate
, s.score_sleep_performance_percentage as sleep_performance_perc
, s.score_sleep_consistency_percentage as sleep_consistency_perc
, s.score_sleep_efficiency_percentage as sleep_efficiency_perc
, s.score sleep_score

, c.score_average_heart_rate as avg_heart_rate
, c.score_max_heart_rate as max_heart_rate
, c.score_kilojoule as kilojoule
, c.score_strain as strain

, sy.score_state as y_score_state
, sy.score_stage_summary_total_in_bed_time_milli as y_in_bed_time
, sy.score_stage_summary_total_no_data_time_milli as y_no_data_time
, sy.score_stage_summary_total_light_sleep_time_milli as y_light_sleep_time
, sy.score_stage_summary_total_slow_wave_sleep_time_milli as y_slow_wave_sleep_time
, sy.score_stage_summary_total_rem_sleep_time_milli as y_rem_sleep_time
, sy.score_stage_summary_sleep_cycle_count as y_sleep_cycle_count
, sy.score_stage_summary_disturbance_count as y_disturbance_count
, sy.score_sleep_needed_baseline_milli as y_sleep_need
, sy.score_sleep_needed_need_from_sleep_debt_milli as y_sleep_need_from_debt
, sy.score_sleep_needed_need_from_recent_strain_milli as y_sleep_need_from_strain
, sy.score_respiratory_rate as y_respiratory_rate
, sy.score_sleep_performance_percentage as y_sleep_performance_perc
, sy.score_sleep_consistency_percentage as y_sleep_consistency_perc
, sy.score_sleep_efficiency_percentage as y_sleep_efficiency_perc
, sy.score y_sleep_score

, cy.score_average_heart_rate as y_avg_heart_rate
, cy.score_max_heart_rate as y_max_heart_rate
, cy.score_kilojoule as y_kilojoule
, cy.score_strain as y_strain

 FROM `msds434-whoop-app.whoopdataset.recovery` r
 left join `msds434-whoop-app.whoopdataset.sleep` s on s.id = r.sleep_id
  left join `msds434-whoop-app.whoopdataset.cycles` c on c.id = r.cycle_id
  left join `msds434-whoop-app.whoopdataset.sleep` sy on date(sy.created_at, "America/Chicago") = date(r.created_at, "America/Chicago") - 1
  left join `msds434-whoop-app.whoopdataset.cycles` cy on date(cy.created_at, "America/Chicago") = date(r.created_at, "America/Chicago") - 1
where s.nap = FALSE
and sy.nap = FALSE
    """
)

whoop_daily = query_job.to_dataframe()

In [7]:
whoop_daily.head().T

,0,1,2,3,4
cycle_id,295732012,270316801,240368447,221623469,263336612
sleep_id,546671256,504894343,456582765,426415802,493626026
recovery_date,2022-11-12,2022-09-06,2022-06-18,2022-04-30,2022-08-18
week_of_year,45,36,24,17,33
day_of_week,7,3,7,7,5
score_state,SCORED,SCORED,SCORED,SCORED,SCORED
recovery_score,1.0,33.0,1.0,90.0,52.0
resting_heart_rate,72.0,63.0,63.0,44.0,46.0
hrv_milli,30.418001,55.321415,35.580547,104.51537,84.505226
spo2_perc,95.833336,97.77778,98.5,97.57895,94.125


In [32]:
# add workouts
# daily data
query_job = client.query(
    """ 
    select 
date(created_at) workout_date
, date(created_at) + 1 tomorrow
, max(extract(hour from created_at AT TIME ZONE 'America/Chicago')*1.0 +
 extract(minute from created_at AT TIME ZONE 'America/Chicago')/60.0) as workout_start_time
, max(score_strain) as workout_strain
, max(score_average_heart_rate) as workout_average_heart_rate
, max(score_max_heart_rate) as workout_max_heart_rate
, max(score_kilojoule) as workout_kilojoule
, max(score_percent_recorded) as percent_recorded
, max(score_zone_duration_zone_one_milli) as zone_one
, max(score_zone_duration_zone_two_milli) as zone_two
, max(score_zone_duration_zone_three_milli) as zone_thee
, max(score_zone_duration_zone_four_milli) as zone_four
, max(score_zone_duration_zone_five_milli) as zone_five
from `whoopdataset.workouts`
group by created_at, date(created_at), date(created_at) + 1, date(created_at) - 1
    """
)

workouts = query_job.to_dataframe()

In [30]:
workouts.head()

,workout_date,workout_start_time,workout_strain,workout_average_heart_rate,workout_max_heart_rate,workout_kilojoule,percent_recorded,zone_one,zone_two,zone_thee,zone_four,zone_five
0,2022-09-05,3.516667,10.5843,157.0,182.0,1269.0616,100.0,14420.0,22109.0,496035.0,805579.0,29800.0
1,2021-08-07,18.583333,12.1878,136.0,160.0,2724.6887,100.0,271126.0,980533.0,2793548.0,9613.0,0.0
2,2021-08-18,13.366667,12.2279,156.0,184.0,1648.3560,100.0,9613.0,57678.0,527756.0,970916.0,116319.0
3,2020-09-11,16.083333,11.0110,147.0,169.0,1688.7795,100.0,31761.0,149965.0,1489060.0,350875.0,0.0
4,2019-09-30,16.366667,9.7933,119.0,169.0,2924.9448,100.0,1536166.0,1606339.0,822873.0,88441.0,0.0


In [13]:
# totals
whoop_daily['total_sleep_time'] = whoop_daily['slow_wave_sleep_time'] + whoop_daily['light_sleep_time'] + whoop_daily['rem_sleep_time']
whoop_daily['y_total_sleep_time'] = whoop_daily['y_slow_wave_sleep_time'] + whoop_daily['y_light_sleep_time'] + whoop_daily['y_rem_sleep_time']

# ACR
whoop_daily['acute_chronic_strain'] = whoop_daily['y_strain'].rolling(window = 7, min_periods=0).mean() / whoop_daily['y_strain'].rolling(window = 30, min_periods=14).mean() # acr

# weekly avgs
whoop_daily['w_strain'] = whoop_daily['y_strain'].rolling(window = 7, min_periods=5).mean() # strain
whoop_daily['w_rtotal_sleep_time'] = whoop_daily['y_rem_sleep_time'].rolling(window = 7, min_periods=5).mean() # total sleep
whoop_daily['w_sleep_start_time_sd'] = whoop_daily['sleep_start_time'].rolling(window = 7, min_periods=5).std().shift(1) # bedtime consitency
whoop_daily['w_slow_wave_sleep_time'] = whoop_daily['y_slow_wave_sleep_time'].rolling(window = 7, min_periods=5).mean()
whoop_daily['w_light_sleep_time'] = whoop_daily['y_light_sleep_time'].rolling(window = 7, min_periods=5).mean()
whoop_daily['w_rem_sleep_time'] = whoop_daily['y_rem_sleep_time'].rolling(window = 7, min_periods=5).mean()
whoop_daily['w_recovery_score'] = whoop_daily['recovery_score'].rolling(window = 7, min_periods=5).mean().shift(1)
whoop_daily['w_hrv_milli'] = whoop_daily['hrv_milli'].rolling(window = 7, min_periods=5).mean().shift(1)
whoop_daily['w_resting_heart_rate'] = whoop_daily['resting_heart_rate'].rolling(window = 7, min_periods=5).mean().shift(1)

In [35]:
whoop_daily_with_workouts = whoop_daily.drop_duplicates("recovery_date").merge(
    workouts.drop_duplicates("workout_date").drop(columns="tomorrow"), 
    left_on="recovery_date",
    right_on= "workout_date",
    how="left"
).merge(
    workouts.add_prefix('y_').drop_duplicates('y_workout_date').drop(columns="y_workout_date"),
    left_on="recovery_date",
    right_on= "y_tomorrow",
    how="left"
)

In [36]:
whoop_daily_with_workouts

,cycle_id,sleep_id,recovery_date,week_of_year,day_of_week,score_state,recovery_score,resting_heart_rate,hrv_milli,spo2_perc,...,y_workout_strain,y_workout_average_heart_rate,y_workout_max_heart_rate,y_workout_kilojoule,y_percent_recorded,y_zone_one,y_zone_two,y_zone_thee,y_zone_four,y_zone_five
0,295732012,546671256,2022-11-12,45,7,SCORED,1.0,72.0,30.418001,95.833336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,270316801,504894343,2022-09-06,36,3,SCORED,33.0,63.0,55.321415,97.777780,...,10.5843,157.0,182.0,1269.0616,100.0,14420.0,22109.0,496035.0,805579.0,29800.0
2,240368447,456582765,2022-06-18,24,7,SCORED,1.0,63.0,35.580547,98.500000,...,8.3609,121.0,149.0,1648.8652,100.0,952662.0,2544605.0,152848.0,0.0,0.0
3,221623469,426415802,2022-04-30,17,7,SCORED,90.0,44.0,104.515370,97.578950,...,15.0359,152.0,185.0,2686.9678,100.0,151926.0,105744.0,84594.0,2027389.0,202837.0
4,263336612,493626026,2022-08-18,33,5,SCORED,52.0,46.0,84.505226,94.125000,...,8.9879,124.0,160.0,1503.7028,100.0,714275.0,826730.0,1128575.0,12496.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1186,209906730,407575234,2022-03-29,13,3,SCORED,95.0,46.0,94.396710,96.205880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1187,182839640,364333223,2022-01-12,2,4,SCORED,58.0,50.0,74.734740,96.375000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1188,308041960,566292459,2022-12-15,50,5,SCORED,70.0,52.0,74.451454,97.833336,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1189,299614407,552820783,2022-11-22,47,3,SCORED,45.0,54.0,65.214660,99.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
pandas_gbq.to_gbq(whoop_daily_with_workouts, 'whoopdataset.whoopmerge', project_id=project_id)

c:\Users\BrettMele\anaconda3\lib\site-packages\google\auth\_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  ):
100%|██████████| 1/1 [00:00<?, ?it/s]
